# Preprocessing audio

### Start execution

In [1]:
import time

In [2]:
start = time.time()

# 1. Setting the environment

In [3]:
!pip install -q resampy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.7 MB/s eta 0:00:00


# 2. Import Libraries

In [4]:
import os
import resampy
import librosa
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split

In [5]:
import warnings
warnings.filterwarnings("ignore")

# 3. Preparation

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
path_general = 'drive/MyDrive/Profesional_Academico/Github_Personal/ML_AI_Contents/09.Deep_Learning/73.Audio_NN_Scratch'

# 4. Read metadata

In [8]:
metadata = pd.read_csv(f'{path_general}/data/UrbanSound8K.csv')

In [9]:
metadata = metadata[['slice_file_name', 'fold', 'class']]
metadata = metadata.rename(columns = {'slice_file_name': 'file_name'})

# 5. Split

In [10]:
metadata_train, metadata_val = train_test_split(metadata, test_size = 0.40, random_state = 42)

In [11]:
metadata_val, metadata_test = train_test_split(metadata_val, test_size = 0.50, random_state = 42)

In [12]:
metadata_train['subset'] = 'train'
metadata_val['subset'] = 'val'
metadata_test['subset'] = 'test'

In [13]:
df_metadata = pd.concat([metadata_train, metadata_val, metadata_test], axis = 0)

In [14]:
df_metadata = df_metadata.reset_index()
df_metadata = df_metadata.drop(columns = ['index'])

In [15]:
df_metadata['file_name'] = 'fold' + df_metadata['fold'].apply(str) + '/' + df_metadata['file_name']

In [16]:
df_metadata = df_metadata.drop(columns = ['fold'])

In [17]:
id_class = {
    'street_music': 1,
    'drilling': 2,
    'children_playing': 3,
    'dog_bark': 4,
    'engine_idling': 5,
    'jackhammer': 6,
    'air_conditioner': 7,
    'siren': 8,
    'car_horn': 9,
    'gun_shot': 10
    }

In [18]:
df_metadata['id_class'] = df_metadata['class']

In [19]:
df_metadata = df_metadata.replace({'id_class': id_class})

In [20]:
df_metadata = df_metadata[['file_name', 'subset', 'id_class']]

In [21]:
df_metadata.to_csv(f'{path_general}/processed/metadata.csv', index = False)

# 6. Process data

In [22]:
for i in range(len(df_metadata)):

  file_name = df_metadata['file_name'].iloc[i]
  subset = df_metadata['subset'].iloc[i]
  id_class = df_metadata['id_class'].iloc[i]

  path_open = f'{path_general}/data/{file_name}'

  file_raw = (file_name.split('/')[1]).split('.')[0]

  path_save = f'{path_general}/processed/{subset}/{id_class}/{file_raw}.npy'

  #------------------------------------------------------------------------------------------

  max_pad_len = 174

  audio, sample_rate = librosa.load(path_open, res_type = 'kaiser_fast')
  mfccs = librosa.feature.mfcc(y = audio, sr = sample_rate, n_mfcc = 40)
  pad_width = max_pad_len - mfccs.shape[1]
  mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

  #------------------------------------------------------------------------------------------

  if mfccs.shape == (40, 174):
    pass

  else:
    print(f'Error: {i}')

  #------------------------------------------------------------------------------------------

  mfccs_def = mfccs.reshape(mfccs.shape[0], mfccs.shape[1], 1)

  with open(path_save, 'wb') as f:
    np.save(f, mfccs_def)

  #------------------------------------------------------------------------------------------

  if (i % 100) == 0:
    print(f'Iteration: {i}')
    print('-'*50)

Iteration: 0
--------------------------------------------------
Iteration: 100
--------------------------------------------------
Iteration: 200
--------------------------------------------------
Iteration: 300
--------------------------------------------------
Iteration: 400
--------------------------------------------------
Iteration: 500
--------------------------------------------------
Iteration: 600
--------------------------------------------------
Iteration: 700
--------------------------------------------------
Iteration: 800
--------------------------------------------------
Iteration: 900
--------------------------------------------------
Iteration: 1000
--------------------------------------------------
Iteration: 1100
--------------------------------------------------
Iteration: 1200
--------------------------------------------------
Iteration: 1300
--------------------------------------------------
Iteration: 1400
--------------------------------------------------
Iterati

### End of execution

In [23]:
end = time.time()

delta = (end - start)

hours = int(delta/3_600)
mins = int((delta - hours*3_600)/60)
secs = int(delta - hours*3_600 - mins*60)

print(f'Hours: {hours}, Minutes: {mins}, Seconds: {secs}')

Hours: 2, Minutes: 0, Seconds: 16
